# 引言

关于PySpark，我们知道它是Python调用Spark的接口，我们可以通过调用Python API的方式来编写Spark程序，它支持了大多数的Spark功能，比如SparkDataFrame、Spark SQL、Streaming、MLlib等等。

![spark](../images/spark.jpg)

[Windows和Jupyter notebook配置pyspark](https://sparkbyexamples.com/pyspark/install-pyspark-in-anaconda-jupyter-notebook/)

在ipython中引入pyspark  
`import findspark`  
`findspark.init()`  
然后就可以自由调用pyspark的API了.

# 基本概念

## 什么是RDD

RDD的全称是 Resilient Distributed Datasets（弹性分布式数据集），这是Spark的一种数据抽象集合，它可以被执行在分布式的集群上进行各种操作，而且有较强的容错机制。RDD可以被分为若干个分区，每一个分区就是一个数据集片段，从而可以支持分布式计算。

## RDD运行时相关的关键名词

- Client：指的是客户端进程，主要负责提交job到Master；

- Job：Job来自于我们编写的程序，Application包含一个或者多个job，job包含各种RDD操作；

- Master：指的是Standalone模式中的主控节点，负责接收来自Client的job，并管理着worker，可以给worker分配任务和资源（主要是driver和executor资源）；

- Worker：指的是Standalone模式中的slave节点，负责管理本节点的资源，同时受Master管理，需要定期给Master回报heartbeat（心跳），启动Driver和Executor；

- Driver：指的是 job（作业）的主进程，一般每个Spark作业都会有一个Driver进程，负责整个作业的运行，包括了job的解析、Stage的生成、调度Task到Executor上去执行；

- Stage：中文名 阶段，是job的基本调度单位，因为每个job会分成若干组Task，每组任务就被称为 Stage；

- Task：任务，指的是直接运行在executor上的东西，是executor上的一个线程；

- Executor：指的是 执行器，顾名思义就是真正执行任务的地方了，一个集群可以被配置若干个Executor，每个Executor接收来自Driver的Task，并执行它（可同时执行多个Task）。

## 什么是DAG

全称是 Directed Acyclic Graph，中文名是有向无环图。Spark就是借用了DAG对RDD之间的关系进行了建模，用来描述RDD之间的因果依赖关系。因为在一个Spark作业调度中，多个作业任务之间也是相互依赖的，有些任务需要在一些任务执行完成了才可以执行的。在Spark调度中就是有DAGscheduler，它负责将job分成若干组Task组成的Stage。

## Spark的部署模式有哪些

主要有local模式、Standalone模式、Mesos模式、YARN模式。

- Standalone： 独立模式，Spark 原生的简单集群管理器， 自带完整的服务， 可单独部署到一个集群中，无需依赖任何其他资源管理系统， 使用 Standalone 可以很方便地搭建一个集群，一般在公司内部没有搭建其他资源管理框架的时候才会使用。
- Mesos：一个强大的分布式资源管理框架，它允许多种不同的框架部署在其上，包括 yarn，由于mesos这种方式目前应用的比较少，这里没有记录mesos的部署方式。
- YARN： 统一的资源管理机制， 在上面可以运行多套计算框架， 如map reduce、storm 等， 根据 driver 在集群中的位置不同，分为 yarn client 和 yarn cluster。

Spark 的运行模式取决于传递给 SparkContext 的 MASTER 环境变量的值， 个别模式还需要辅助的程序接口来配合使用，目前支持的 Master 字符串及 URL 包括：

|MasterURL	|Meaning|
|:--|:--|
|local	|在本地运行，只有一个工作进程，无并行计算能力
|local[K]|	在本地运行，有 K 个工作进程，通常设置 K 为机器的CPU 核心数量
|local[*]|	在本地运行，工作进程数量等于机器的 CPU 核心数量。
|spark://HOST:PORT	|以 Standalone 模式运行，这是 Spark 自身提供的集群运行模式，默认端口号: 7077
|mesos://HOST:PORT	|在 Mesos 集群上运行，Driver 进程和 Worker 进程运行在 Mesos 集群上，部署模式必须使用固定值:--deploy-mode cluster
|yarn-client|	在 Yarn 集群上运行，Driver 进程在本地， Work 进程在 Yarn 集群上， 部署模式必须使用固定值:--deploy-modeclient。Yarn 集群地址必须在HADOOP_CONF_DIRorYARN_CONF_DIR 变量里定义。


## Shuffle操作是什么

Shuffle指的是数据从Map端到Reduce端的数据传输过程，Shuffle性能的高低直接会影响程序的性能。因为Reduce task需要跨节点去拉在分布在不同节点上的Map task计算结果，这一个过程是需要有磁盘IO消耗以及数据网络传输的消耗的，所以需要根据实际数据情况进行适当调整。另外，Shuffle可以分为两部分，分别是Map阶段的数据准备与Reduce阶段的数据拷贝处理，在Map端我们叫Shuffle Write，在Reduce端我们叫Shuffle Read。

## 什么是惰性执行

这是RDD的一个特性，在RDD中的算子可以分为Transform算子和Action算子，其中Transform算子的操作都不会真正执行，只会记录一下依赖关系，直到遇见了Action算子，在这之前的所有Transform操作才会被触发计算，这就是所谓的惰性执行。

- Transform算子:map、flatMap、filter、distinct、reduceByKey、mapPartitions、sortBy
- Action算子:collect、collectAsMap、reduce、countByKey、take、first

# 常用函数

[官方API文档](https://spark.apache.org/docs/latest/api/python/reference/index.html)

In [9]:
import pyspark 
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("test").setMaster("local[4]")
sc = SparkContext(conf=conf)

print("spark version:",pyspark.__version__)
rdd = sc.parallelize(["hello","spark"])
print(rdd.reduce(lambda x,y:x+' '+y))
sc.stop()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=test, master=local[4]) created by __init__ at C:\Users\IKAS\AppData\Local\Temp\ipykernel_20392\526825949.py:4 

In [11]:
sc.stop()

In [7]:
#在jupyter中使用pyspark
import findspark
#findspark.init()

In [8]:
spark_home = "D:\\Anaconda\\Lib\\site-packages\\pyspark"
python_path = "D:\\Anaconda\\python"
findspark.init(spark_home,python_path)

In [12]:
import os
import pyspark
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName("test_SamShare").setMaster("local[4]")
sc = SparkContext(conf=conf)

# 使用 parallelize方法直接实例化一个RDD
rdd = sc.parallelize(range(1,11),4) # 这里的 4 指的是分区数量
rdd.take(10)

TypeError: 'JavaPackage' object is not callable

## Transform算子解析

以下的操作由于是Transform操作，因为我们需要在最后加上一个collect算子用来触发计算。

### map

In [15]:
# 1. map: 和python差不多，map转换就是对每一个元素进行一个映射
rdd = sc.parallelize(range(1, 11), 4)
rdd_map = rdd.map(lambda x: x*2)
print("原始数据：", rdd.collect())
print("扩大2倍：", rdd_map.collect())

原始数据： [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
扩大2倍： [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]


### flatMap

In [13]:
# 2. flatMap: 这个相比于map多一个flat（压平）操作，顾名思义就是要把高维的数组变成一维
rdd2 = sc.parallelize(["hello SamShare", "hello PySpark"])
print("原始数据：", rdd2.collect())
print("直接split之后的map结果：", rdd2.map(lambda x: x.split(" ")).collect())
print("直接split之后的flatMap结果：", rdd2.flatMap(lambda x: x.split(" ")).collect())

原始数据： ['hello SamShare', 'hello PySpark']
直接split之后的map结果： [['hello', 'SamShare'], ['hello', 'PySpark']]
直接split之后的flatMap结果： ['hello', 'SamShare', 'hello', 'PySpark']


### filter

In [14]:
# 3. filter: 过滤数据
rdd = sc.parallelize(range(1, 11), 4)
print("原始数据：", rdd.collect())
print("过滤奇数：", rdd.filter(lambda x: x % 2 == 0).collect())

原始数据： [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
过滤奇数： [2, 4, 6, 8, 10]


### distinct

In [15]:
# 4. distinct: 去重元素
rdd = sc.parallelize([2, 2, 4, 8, 8, 8, 8, 16, 32, 32])
print("原始数据：", rdd.collect())
print("去重数据：", rdd.distinct().collect())

原始数据： [2, 2, 4, 8, 8, 8, 8, 16, 32, 32]
去重数据： [4, 8, 16, 32, 2]


### reduceByKey

In [16]:
# 5. reduceByKey: 根据key来映射数据
from operator import add
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
print("原始数据：", rdd.collect())
print("原始数据：", rdd.reduceByKey(add).collect())

原始数据： [('a', 1), ('b', 1), ('a', 1)]
原始数据： [('b', 1), ('a', 2)]


### mapPartitions

In [17]:
# 6. mapPartitions: 根据分区内的数据进行映射操作
rdd = sc.parallelize([1, 2, 3, 4], 2)
def f(iterator):
    yield sum(iterator)
print(rdd.collect())
print(rdd.mapPartitions(f).collect())

[1, 2, 3, 4]
[3, 7]


### sortBy

In [18]:
# 7. sortBy: 根据规则进行排序
tmp = [('a', 1), ('b', 2), ('1', 3), ('d', 4), ('2', 5)]
print(sc.parallelize(tmp).sortBy(lambda x: x[0]).collect())
print(sc.parallelize(tmp).sortBy(lambda x: x[1]).collect())

[('1', 3), ('2', 5), ('a', 1), ('b', 2), ('d', 4)]
[('a', 1), ('b', 2), ('1', 3), ('d', 4), ('2', 5)]


### subtract

In [19]:
# 8. subtract: 数据集相减, Return each value in self that is not contained in other.
x = sc.parallelize([("a", 1), ("b", 4), ("b", 5), ("a", 3)])
y = sc.parallelize([("a", 3), ("c", None)])
print(sorted(x.subtract(y).collect()))

[('a', 1), ('b', 4), ('b', 5)]


### union

In [20]:
# 9. union: 合并两个RDD
rdd = sc.parallelize([1, 1, 2, 3])
print(rdd.union(rdd).collect())

[1, 1, 2, 3, 1, 1, 2, 3]


### intersection

In [21]:
# 10. intersection: 取两个RDD的交集，同时有去重的功效
rdd1 = sc.parallelize([1, 10, 2, 3, 4, 5, 2, 3])
rdd2 = sc.parallelize([1, 6, 2, 3, 7, 8])
print(rdd1.intersection(rdd2).collect())

[1, 2, 3]


### cartesian

In [22]:
# 11. cartesian: 生成笛卡尔积
rdd = sc.parallelize([1, 2])
print(sorted(rdd.cartesian(rdd).collect()))

[(1, 1), (1, 2), (2, 1), (2, 2)]


### zip

In [23]:
# 12. zip: 拉链合并，需要两个RDD具有相同的长度以及分区数量
x = sc.parallelize(range(0, 5))
y = sc.parallelize(range(1000, 1005))
print(x.collect())
print(y.collect())
print(x.zip(y).collect())

[0, 1, 2, 3, 4]
[1000, 1001, 1002, 1003, 1004]
[(0, 1000), (1, 1001), (2, 1002), (3, 1003), (4, 1004)]


### zipWithIndex

In [24]:
# 13. zipWithIndex: 将RDD和一个从0开始的递增序列按照拉链方式连接。
rdd_name = sc.parallelize(["LiLei", "Hanmeimei", "Lily", "Lucy", "Ann", "Dachui", "RuHua"])
rdd_index = rdd_name.zipWithIndex()
print(rdd_index.collect())

[('LiLei', 0), ('Hanmeimei', 1), ('Lily', 2), ('Lucy', 3), ('Ann', 4), ('Dachui', 5), ('RuHua', 6)]


### groupByKey

In [25]:
# 14. groupByKey: 按照key来聚合数据
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
print(rdd.collect())
print(sorted(rdd.groupByKey().mapValues(len).collect()))
print(sorted(rdd.groupByKey().mapValues(list).collect()))

[('a', 1), ('b', 1), ('a', 1)]
[('a', 2), ('b', 1)]
[('a', [1, 1]), ('b', [1])]


### sortByKey

In [26]:
# 15. sortByKey:
tmp = [('a', 1), ('b', 2), ('1', 3), ('d', 4), ('2', 5)]
print(sc.parallelize(tmp).sortByKey(True, 1).collect())

[('1', 3), ('2', 5), ('a', 1), ('b', 2), ('d', 4)]


### join

In [27]:
# 16. join:
x = sc.parallelize([("a", 1), ("b", 4)])
y = sc.parallelize([("a", 2), ("a", 3)])
print(sorted(x.join(y).collect()))

[('a', (1, 2)), ('a', (1, 3))]


### leftOuterJoin/rightOuterJoin

In [28]:
# 17. leftOuterJoin/rightOuterJoin
x = sc.parallelize([("a", 1), ("b", 4)])
y = sc.parallelize([("a", 2)])
print(sorted(x.leftOuterJoin(y).collect()))

[('a', (1, 2)), ('b', (4, None))]


## Action算子解析

### collect

In [29]:
# 1. collect: 指的是把数据都汇集到driver端，便于后续的操作
rdd = sc.parallelize(range(0, 5))
rdd_collect = rdd.collect()
print(rdd_collect)

[0, 1, 2, 3, 4]


### first

In [30]:
# 2. first: 取第一个元素
sc.parallelize([2, 3, 4]).first()

2

### collectAsMap

In [31]:
# 3. collectAsMap: 转换为dict，使用这个要注意了，不要对大数据用，不然全部载入到driver端会爆内存
m = sc.parallelize([(1, 2), (3, 4)]).collectAsMap()
m

{1: 2, 3: 4}

### reduce

In [32]:
# 4. reduce: 逐步对两个元素进行操作
rdd = sc.parallelize(range(10),5)
print(rdd.reduce(lambda x,y:x+y))

45


### countByKey/countByValue

In [33]:
# 5. countByKey/countByValue:
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
print(sorted(rdd.countByKey().items()))
print(sorted(rdd.countByValue().items()))

[('a', 2), ('b', 1)]
[(('a', 1), 2), (('b', 1), 1)]


### take

In [34]:
# 6. take: 相当于取几个数据到driver端
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
print(rdd.take(5))

[('a', 1), ('b', 1), ('a', 1)]


### saveAsTextFile

In [ ]:
# 7. saveAsTextFile: 保存rdd成text文件到本地
text_file = "./data/rdd.txt"
rdd = sc.parallelize(range(5))
rdd.saveAsTextFile(text_file)

### takeSample

In [ ]:
# 8. takeSample: 随机取数
rdd = sc.textFile("./test/data/hello_samshare.txt", 4)  # 这里的 4 指的是分区数量
rdd_sample = rdd.takeSample(True, 2, 0)  # withReplacement 参数1：代表是否是有放回抽样
rdd_sample

### foreach

In [35]:
# 9. foreach: 对每一个元素执行某种操作，不生成新的RDD
rdd = sc.parallelize(range(10), 5)
accum = sc.accumulator(0)
rdd.foreach(lambda x: accum.add(x))
print(accum.value)

45


# Spark SQL使用

这个模块是Spark中用来处理结构化数据的，提供一个叫SparkDataFrame的东西并且自动解析为分布式SQL查询数据。我们之前用过Python的Pandas库，也大致了解了DataFrame，这个其实和它没有太大的区别，只是调用的API可能有些不同罢了。

# Spark调优思路

## 开发习惯调优

### 尽可能复用同一个RDD，避免重复创建，并且适当持久化数据

这种开发习惯是需要我们对于即将要开发的应用逻辑有比较深刻的思考，并且可以通过code review来发现的，讲白了就是要记得我们创建过啥数据集，可以复用的尽量广播（broadcast）下，能很好提升性能。

### 尽量避免使用低性能算子

shuffle类算子算是低性能算子的一种代表，所谓的shuffle类算子，指的是会产生shuffle过程的操作，就是需要把各个节点上的相同key写入到本地磁盘文件中，然后其他的节点通过网络传输拉取自己需要的key，把相同key拉到同一个节点上进行聚合计算，这种操作必然就是有大量的数据网络传输与磁盘读写操作，性能往往不是很好的。

### 尽量使用高性能算子

### 广播大变量

如果我们有一个数据集很大，并且在后续的算子执行中会被反复调用，那么就建议直接把它广播（broadcast）一下。当变量被广播后，会保证每个executor的内存中只会保留一份副本，同个executor内的task都可以共享这个副本数据。如果没有广播，常规过程就是把大变量进行网络传输到每一个相关task中去，这样子做，一来频繁的网络数据传输，效率极其低下；二来executor下的task不断存储同一份大数据，很有可能就造成了内存溢出或者频繁GC，效率也是极其低下的。

In [102]:
rdd1 = sc.parallelize([('A1', 11), ('A2', 12), ('A3', 13), ('A4', 14)])
rdd1_broadcast = sc.broadcast(rdd1.collect())
print(rdd1.collect())
print(rdd1_broadcast.value)

[('A1', 11), ('A2', 12), ('A3', 13), ('A4', 14)]
[('A1', 11), ('A2', 12), ('A3', 13), ('A4', 14)]


## 资源参数调优

如果要进行资源调优，我们就必须先知道Spark运行的机制与流程。

![spark](spark.jpg)

### num-executors

指的是执行器的数量，数量的多少代表了并行的stage数量（假如executor是单核的话），但也并不是越多越快，受你集群资源的限制，所以一般设置50-100左右吧。

### executor-memory

这里指的是每一个执行器的内存大小，内存越大当然对于程序运行是很好的了，但是也不是无节制地大下去，同样受我们集群资源的限制。假设我们集群资源为500core，一般1core配置4G内存，所以集群最大的内存资源只有2000G左右。num-executors x executor-memory 是不能超过2000G的，但是也不要太接近这个值，不然的话集群其他同事就没法正常跑数据了，一般我们设置4G-8G。

### executor-cores

这里设置的是executor的CPU core数量，决定了executor进程并行处理task的能力。

### driver-memory

设置driver的内存，一般设置2G就好了。但如果想要做一些Python的DataFrame操作可以适当地把这个值设大一些。

### driver-cores

与executor-cores类似的功能。

### spark.default.parallelism

设置每个stage的task数量。一般Spark任务我们设置task数量在500-1000左右比较合适，如果不去设置的话，Spark会根据底层HDFS的block数量来自行设置task数量。有的时候会设置得偏少，这样子程序就会跑得很慢，即便你设置了很多的executor，但也没有用。